In [11]:
import pandas as pd
import numpy as np
import math
import logging
import importlib

import sys
import os

import matplotlib.pyplot as plt
import mlflow

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from mlflow import log_metric, log_param, log_artifact
from sklearn.ensemble import StackingClassifier
from inspect import getsource
import inspect

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.pipeline import Pipeline

import tensorflow as tf

import tempfile


In [2]:
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from data.load_data import load_data
from data.split_data import split_data
from data.clean_data import DataCleaner
from data.string_toInt_encoder import StringToIntEncoder
from data.oversample_minority import OversampleMinority
from data.undersample_majority import UndersampleMajority
from data.featur_target_pipline import FeatureTargetPipeline
from data.target_binarizer import TargetBinarizer
from data.dummies_encoder import SafeOneHotEncoder
from data.artist_popularity import ArtistPopularityEncoder
from data.balance_simpler import BalancedResampler


from predict.acuuracy import ModelEvaluator

from train.mlflow_experiment_runner import MLflowExperimentRunner
# from train.model_evaluator import ModelEvaluator


import data.load_data
import data.split_data
import data.clean_data
import data.string_toInt_encoder
import data.oversample_minority
import data.undersample_majority
import data.featur_target_pipline
import data.dummies_encoder
import data.artist_popularity
import data.balance_simpler

import predict.acuuracy

# import train.model_evaluator
import train.mlflow_experiment_runner


importlib.reload(data.load_data)
importlib.reload(data.split_data)
importlib.reload(data.clean_data)
importlib.reload(data.string_toInt_encoder)
importlib.reload(data.oversample_minority)
importlib.reload(data.undersample_majority)
importlib.reload(data.dummies_encoder)
importlib.reload(data.featur_target_pipline)
importlib.reload(data.target_binarizer)
importlib.reload(data.artist_popularity)
importlib.reload(data.balance_simpler)

# importlib.reload(train.model_evaluator)
importlib.reload(train.mlflow_experiment_runner)


importlib.reload(predict.acuuracy)



<module 'predict.acuuracy' from '/Users/level3/mlops_spotify/spotify-1million/src/predict/acuuracy.py'>

In [3]:
data_folder = './data'
df = load_data(data_folder)

Loading data...
Dataset URL: https://www.kaggle.com/datasets/amitanshjoshi/spotify-1million-tracks
Download complete.
Loading CSV data...
Data loaded successfully.


In [4]:
target = df['popularity']
y = TargetBinarizer(threshold=50).fit_transform(target)
X = df.drop(columns='popularity')

X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y)

In [6]:
base_preprocessor = Pipeline([
    ('artist_encoder', ArtistPopularityEncoder()),
    ('data_cleaner', DataCleaner()),
    ('encoder', SafeOneHotEncoder(columns=['genre', 'key', 'time_signature'])),
    ('scaler', StandardScaler()),
])

In [12]:
# In your DLExperimentWrapper
from tensorflow.keras.mixed_precision import Policy
policy = Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

In [14]:
# Example of how to use the MLflowExperimentRunner class with your experiment configuration

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np


# 1. Create an instance of the experiment runner
runner = MLflowExperimentRunner(
    experiment_name="Spotify_Popularity",
    tracking_uri="http://localhost:5000",
    evaluator_class=ModelEvaluator  # Set default evaluator class
)

# 2. Define your experiment configurations
experiments = [
    {
        "model": XGBClassifier(),
        "sampler": BalancedResampler,  # Your custom resampler class
        "preprocessor": base_preprocessor,  # Your preprocessing pipeline
        "X_train": X_train,
        "y_train": y_train,
        "X_val": X_val,
        "y_val": y_val,
        "model_params": {
            'n_estimators': 300,
            'max_depth': 8,
            'learning_rate': 0.1,
            'scale_pos_weight': 1.9
        },
        "sampler_params": {},
        "dataset_version": "2024-03-v2",
        "input_example": X_train.sample(5),
        "metadata": {
            "business_impact": "high",
            "owner": "data-team",
            "description": "XGBoost with class balancing"
        },
        "tags": {
            "stage": "production-candidate",
            "data_source": "spotify-api"
        },
        # "registered_model_name": "SpotifyPopularityClassifier",  # Uncomment to register
        "log_artifacts": True
    },
    # You can add more experiment configurations here
    # {
    #     "model": RandomForestClassifier(),
    #     "preprocessor": base_preprocessor,  # Your preprocessing pipeline
    #     "sampler": BalancedResampler,
    #     "X_train": X_train,
    #     "y_train": y_train, 
    #     "X_val": X_val,
    #     "y_val": y_val,
    #     "model_params": {
    #         'n_estimators': 200,
    #         'max_depth': 10,
    #         'min_samples_split': 5
    #     },
    #     "sampler_params": {},
    #     "metadata": {
    #         "description": "Random Forest baseline model"
    #     },
    #     "tags": {
    #         "stage": "baseline"
    #     }
    # }
]

# 3. Run all experiments at once
results = runner.run_experiments(experiments)
for i, result in enumerate(results):
    print(f"Experiment {i+1} results: {result}")

# Or run them individually
# for exp in experiments:
#     result = runner.run_experiment(**exp)
#     print(f"Experiment results: {result}")

/Users/level3/mlops_spotify/spotify-1million/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run gifted-mule-824 at: http://localhost:5000/#/experiments/400003216354571829/runs/2221aa75aba84748a082e0defd6e5c4e
🧪 View experiment at: http://localhost:5000/#/experiments/400003216354571829
Experiment 1 results: {'accuracy': 0.8359727531399994, 'precision': 0.20635829872547615, 'recall': 0.8972602739726028, 'f1': 0.3355454651298172, 'roc_auc': np.float64(0.9378699276874045)}
